<a href="https://colab.research.google.com/github/anilbhatt1/EVA4P1_S15A_Depth_FG_Detection_DataPrep/blob/master/Statistics_FG_BG_Mask_Depth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! nvidia-smi

Fri May 29 12:10:14 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
from zipfile import ZipFile
import zipfile
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
import io
import skimage
from skimage.transform import resize
from itertools import groupby
import cv2
from tqdm.auto import tqdm
from pathlib import Path
from time import time
from datetime import datetime 
import gc
sns.set()
import os

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Statistics of FG_BG Images

In [0]:
#!unzip -q "/content/gdrive/My Drive/EVA4P1_S15/FG_BG/FG_BG.zip"
FG_BG_zip =  ZipFile("/content/gdrive/My Drive/EVA4P1_S15/FG_BG_Mask_Depth/FG_BG_400K.zip", 'r')

In [0]:
all_files = [info.filename for info in FG_BG_zip.infolist() if not info.is_dir()]

In [0]:
n = 0
s = np.zeros(3)
sq = np.zeros(3)
for img_file in all_files[0:]:
    if img_file.endswith('jpg'):
        imgdata = FG_BG_zip.read(img_file)
        data = Image.open(io.BytesIO(imgdata))
        x = np.array(data)/255
        s += x.sum(axis=(0,1))
        sq += np.sum(np.square(x),axis=(0,1))
        n += x.shape[0]*x.shape[1]
mu = s/n
std =np.sqrt((sq/n - np.square(mu)))
print(mu,sq/n,std,n)

[0.56670278 0.49779153 0.43632878] [0.38389994 0.30871084 0.25551239] [0.25049532 0.2468085  0.25520498] 14745600000


## Statistics of FG_BG_Mask Images

In [0]:
FG_BG_Mask_zip =  ZipFile("/content/gdrive/My Drive/EVA4P1_S15/FG_BG_Mask_Depth/FG_BG_Mask_400K.zip", 'r')

In [0]:
all_files = [info.filename for info in FG_BG_Mask_zip.infolist() if not info.is_dir()]

In [0]:
all_files[0]

'FG_BG_Mask_400K/Img_fg_bg_mask389180.jpg'

In [0]:
n = 0
s = np.zeros(1)
sq = np.zeros(1)
for img_file in all_files[0:]:
    if img_file.endswith('jpg'):
        imgdata = FG_BG_Mask_zip.read(img_file)
        data = Image.open(io.BytesIO(imgdata))
        x = np.array(data)/255
        s += x.sum(axis=(0,1))
        sq += np.sum(np.square(x),axis=(0,1))
        n += x.shape[0]*x.shape[1]
mu = s/n
std =np.sqrt((sq/n - np.square(mu)))
print(mu,sq/n,std,n)

[0.20249742] [0.20069515] [0.39961225] 14745600000


## Statistics of FG_BG_Depth Images

In [0]:
FG_BG_Depth_zip =  ZipFile("/content/gdrive/My Drive/EVA4P1_S15/FG_BG_Mask_Depth/FG_BG_Depth_400K.zip", 'r')
all_files = [info.filename for info in FG_BG_Depth_zip.infolist() if not info.is_dir()]
print(all_files[0])
print(len(all_files))

FG_BG_Depth_0_400K/Img_fg_bg_214327_depth.jpg
400000


In [0]:
n = 0
s = np.zeros(1)
sq = np.zeros(1)
for img_file in all_files[0:]:
    if img_file.endswith('jpg'):
        imgdata = FG_BG_Depth_zip.read(img_file)
        data = Image.open(io.BytesIO(imgdata))
        x = np.array(data)/255
        s += x.sum(axis=(0,1))
        sq += np.sum(np.square(x),axis=(0,1))
        n += x.shape[0]*x.shape[1]
mu = s/n
std =np.sqrt((sq/n - np.square(mu)))
print(mu,sq/n,std,n)

[0.32939295] [0.17065376] [0.24930712] 16000000000


## Statistics of BG and its Flip Images

In [10]:
path = '/content/gdrive/My Drive/EVA4P1_S15/FG_BG_Mask_Depth/BG_and_Its_Flip/'
n = 0
s = np.zeros(3)
sq = np.zeros(3)
for img_file in os.listdir('/content/gdrive/My Drive/EVA4P1_S15/FG_BG_Mask_Depth/BG_and_Its_Flip/'):
    if img_file.endswith('jpg'):
        data = Image.open(f'{path}{img_file}')
        x = np.array(data)/255
        s += x.sum(axis=(0,1))
        sq += np.sum(np.square(x),axis=(0,1))
        n += x.shape[0]*x.shape[1]
mu = s/n
std =np.sqrt((sq/n - np.square(mu)))
print(mu,sq/n,std,n)

[0.58245822 0.51269352 0.43691653] [0.39807445 0.32199507 0.25542021] [0.24252189 0.24318804 0.25401604] 7741440
